In [ ]:
# awi spawn emr instance -- python code in s3://python_code.py -master-node  m5.2xlarge -worker-node m5.xlarge -worker-node 10

In [ ]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=56206b67aa9bceadbb27b6e5fbf6e409c8bb9e0557766f74579848063cd1de0b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

In [ ]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
df = spark.read.csv('click_stream.csv', header=True, inferSchema=True)

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.show(5)

+------------------+-----------+------------+--------------------+--------------------+--------------------+---------+------------------+------------------+-------------------+---------------+--------+--------------------+-----------+----------------+-------+----------+-----------+----------+-------------+-------------------------+--------------+----------+--------------------+------------+-------------+------------+--------------+------------+-----------------+-----------+-----+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+--------

In [ ]:
df.printSchema()

root
 |-- hotel_id: long (nullable = true)
 |-- uuid: string (nullable = true)
 |-- profile_type: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- visitor_id: string (nullable = true)
 |-- user_mcid: string (nullable = true)
 |-- page_name: string (nullable = true)
 |-- meta_activity_name: string (nullable = true)
 |-- meta_act_cntnt: long (nullable = true)
 |-- meta_timezone: string (nullable = true)
 |-- meta_user_agent: string (nullable = true)
 |-- lob_name: string (nullable = true)
 |-- pg_nm_omniture: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- dvc_g_city: string (nullable = true)
 |-- os: string (nullable = true)
 |-- os_version: string (nullable = true)
 |-- app_version: string (nullable = true)
 |-- query_city: string (nullable = true)
 |-- query_country: string (nullable = true)
 |-- query_travel_purpose_optd: string (nullable = true)
 |-- travel_purpose: string (nullable = true)
 |-- query_type: string (nullable = true)
 |--

In [ ]:
# no of rows

df.count()

10409

In [ ]:
# describe

df.describe().show()

+-------+--------------------+-----------+------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+-------------+-----------------+--------+--------------------+-----------+----------+-------+------------------+------------------+----------+-------------+-------------------------+--------------+----------+--------------------+---------------+------------------+-------------------+------------------+------------------+------------------+------------------+-------------+------------------+------------------+------------------+-----------------+------------------+------+------------------+-----------------+-----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------

In [ ]:
df.columns

['hotel_id',
 'uuid',
 'profile_type',
 'device_id',
 'visitor_id',
 'user_mcid',
 'page_name',
 'meta_activity_name',
 'meta_act_cntnt',
 'meta_timezone',
 'meta_user_agent',
 'lob_name',
 'pg_nm_omniture',
 'device_type',
 'dvc_g_city',
 'os',
 'os_version',
 'app_version',
 'query_city',
 'query_country',
 'query_travel_purpose_optd',
 'travel_purpose',
 'query_type',
 'query_value',
 'checkin_date',
 'checkout_date',
 'total_adults',
 'total_children',
 'total_guests',
 'total_room_nights',
 'total_rooms',
 'ap',
 'is_multi_room',
 'length_of_stay',
 'disp_sell_amt',
 'disp_sell_amt_dis',
 'is_free_cncl',
 'htl_meal_plan_code',
 'is_pah',
 'usr_wal_bonus_bal',
 'usr_wal_plus_bal',
 'usr_wal_real_bal',
 'room_code',
 'room_name',
 'room_type',
 'roomstayqualifier',
 'prc_aaj_ka_bhao_flg',
 'prc_absrpn_amt',
 'prc_amt_chrgd',
 'prc_commission',
 'prc_disp_amt_w_dis_srvr',
 'prc_disp_amt_wo_dis_srvr',
 'prc_div_fac',
 'prc_extrapax_amt',
 'prc_hotelier_promotion',
 'prc_htl_base_prc',

# How to select rows?

In [ ]:
df.select('htl_city_code', 'country_code').show()

+-------------+------------+
|htl_city_code|country_code|
+-------------+------------+
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
+-------------+------------+
only showing top 20 rows



In [ ]:
# select rows which are not null
df.select('htl_city_code', 'country_code').where(df.htl_city_code.isNotNull() & df.country_code.isNotNull()).show() #.collect()

+-------------+------------+
|htl_city_code|country_code|
+-------------+------------+
|  high_to_low|       false|
|  high_to_low|       false|
+-------------+------------+



In [ ]:
df.createOrReplaceTempView('click_stream') # CTE of the entire dataframe on top of which I can type SQL queries

spark.sql('select htl_city_code, country_code from click_stream where country_code is not NULL').show()




+-------------+------------+
|htl_city_code|country_code|
+-------------+------------+
|  high_to_low|       false|
|  high_to_low|       false|
+-------------+------------+



In [ ]:
# show and collect both work on entire dataset, but show only shows top x rows

# Missing Values

In [ ]:
# df = df.dropna()

null_counts = df.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

total_rows = df.count()

null_percentage = null_counts.select([(sf.col(c) / total_rows * 100).alias(c) for c in null_counts.columns])
null_percentage.show()

+--------+----+------------+---------+----------+---------+---------+------------------+--------------+-------------+---------------+--------+--------------+-----------+----------+---+----------+-----------+----------+-------------+-------------------------+--------------+----------+-----------+------------+-------------+------------+--------------+------------+-----------------+-----------+---+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+-----------+------------------+--------------+-------+-----------+---------------+------+-----

In [ ]:
df.select(sf.mean((df['total_children']))).collect()

[Row(avg(total_children)=0.11975524475524474)]

In [ ]:
df.select(sf.mean((df['total_children']))).collect()[0]

Row(avg(total_children)=0.11975524475524474)

In [ ]:
df.select(sf.mean((df['total_children']))).collect()[0][0]

0.11975524475524474

In [ ]:
mean_value = df.select(sf.mean((df['total_children']))).collect()[0][0]

# mode_value = df.select(sf.mode((df['total_children']))).collect()[0][0]

df = df.na.fill({'total_children': mean_value})

# Let's try groupby

In [ ]:
df.groupby('htl_city_code').agg(sf.mean('total_adults').alias('avg_adults_in_city')).show()

# df.groupby('hotel_city_codde')['total_adults'].mean()

+-------------+------------------+
|htl_city_code|avg_adults_in_city|
+-------------+------------------+
|          DDM| 4.119658119658119|
|          DBG|1.8141592920353982|
|         DEAS|               2.0|
|          DBV|1.3008474576271187|
|         DBV1|               1.0|
|          DED| 2.204138279081504|
|  high_to_low|              NULL|
|         DAVO| 4.636363636363637|
|          DEL|2.0989623366641044|
|         DCAR|               2.0|
|          DBI|2.1989389920424403|
+-------------+------------------+



In [ ]:
df.select('htl_city_code', 'country_code').show()

+-------------+------------+
|htl_city_code|country_code|
+-------------+------------+
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
|         DAVO|        NULL|
|          DEL|        NULL|
+-------------+------------+
only showing top 20 rows



# How to use filter

In [ ]:
df.filter(df.total_adults.isin([2, 5, 8, 10])).show()

+------------------+-----------+------------+--------------------+--------------------+--------------------+---------+------------------+------------------+-------------------+---------------+--------+--------------------+-----------+----------+-------+----------+-----------+----------+-------------+-------------------------+--------------+----------+--------------------+------------+-------------+------------+--------------+------------+-----------------+-----------+-----+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+-----------+--

In [ ]:
df.filter(df['total_adults'] == 2).groupBy('total_children').agg({'total_room_nights': 'mean'}).withColumnRenamed('avg(total_room_nights)', 'room_per_night').show()

+-------------------+------------------+
|     total_children|    room_per_night|
+-------------------+------------------+
|                0.0|1.6313566206336312|
|0.11975524475524475| 1.018348623853211|
|                1.0|1.9615384615384615|
|                4.0|               1.0|
|                3.0|               2.0|
|                2.0|1.7131147540983607|
+-------------------+------------------+



In [ ]:
df = df.withColumn('couple', sf.when(df.total_adults > 2, 'not-couple').otherwise('couple'))
df.show()

+------------------+-----------+------------+--------------------+--------------------+--------------------+---------+------------------+------------------+-------------------+---------------+--------+--------------------+-----------+----------------+-------+----------+-----------+----------+-------------+-------------------------+--------------+----------+--------------------+------------+-------------+------------+--------------+------------+-----------------+-----------+-----+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+--------

In [ ]:
df.withColumnRenamed('couple', '2-adults').show()

+------------------+-----------+------------+--------------------+--------------------+--------------------+---------+------------------+------------------+-------------------+---------------+--------+--------------------+-----------+----------------+-------+----------+-----------+----------+-------------+-------------------------+--------------+----------+--------------------+------------+-------------+------------+--------------+------------+-----------------+-----------+-----+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+--------

In [ ]:
# udf
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType


# in pyspark we've something called udf
def custom_python_function(value):
  if value == 'not-couple':
    return 0
  else:
    return 1
  # return value * 2

udf_custom_function =udf(custom_python_function, StringType())


df.withColumn('mod_total_adults', udf_custom_function(sf.col('couple'))).show()

+------------------+-----------+------------+--------------------+--------------------+--------------------+---------+------------------+------------------+-------------------+---------------+--------+--------------------+-----------+----------------+-------+----------+-----------+----------+-------------+-------------------------+--------------+----------+--------------------+------------+-------------+------------+--------------+------------+-----------------+-----------+-----+-------------+--------------+-------------+-----------------+------------+------------------+------+-----------------+----------------+----------------+---------+---------+---------+-----------------+-------------------+--------------+-------------+--------------+-----------------------+------------------------+-----------+----------------+----------------------+----------------+--------------+-----------+----------+------------------+-------------------+--------------------+---------------+-----------+--------

In [ ]:
df.toPandas()

# primary stuff in module test
1. drift
2. experiment tracking
3. github
4. docker